## 問題

https://twitter.com/e869120/status/1381739128291614720

## 解説

https://twitter.com/e869120/status/1382101716066127872

In [1]:
#r "nuget: FSharpPlus"

Installed Packages FSharpPlus, 1.3.2

In [2]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus

In [3]:
type [<Struct>] NodeId = NodeId of int
with
  static member op_Implicit i = NodeId i
  static member id (NodeId i) = i

type Edge = {
  Nodes: NodeId * NodeId
  Cost: int
} with
  static member tryGetOtherNode node edge =
    let x, y = edge.Nodes
    if node = x then Some y elif node = y then Some x else None

let allNodeSet edges = edges |> Seq.collect (fun e -> let (x, y) = e.Nodes in [x; y]) |> Seq.distinct |> set

In [7]:
let dijkstra edges nodeSet startNode =
  let rec loop results edges restNodeSet =
    if restNodeSet |> Set.isEmpty then results else
    let (n, _) as result =
      results |> Seq.collect (fun (n, c) ->
        edges |> Seq.choose (fun e -> e |> Edge.tryGetOtherNode n |> Option.map (fun n' -> n', e.Cost + c)))
      |> Seq.minBy snd
    let edges = edges |> filter (Edge.tryGetOtherNode n >> Option.forall (fun n' -> restNodeSet |> Set.contains n'))
    restNodeSet |> Set.remove n |> loop (result::results) edges
  nodeSet |> Set.remove startNode |> loop [startNode, 0] edges

In [13]:
let solve ABCs =
  let edges = ABCs |> Seq.map (fun (a, b, c) -> { Nodes = NodeId a, NodeId b; Cost = c }) |> Seq.toList
  let nodeSet = edges |> allNodeSet
  let N = nodeSet |> Set.count
  nodeSet |> Seq.map (fun n ->
    // 経由地nに寄った上での最短距離は、始点からnの最短距離 + 終点からnの最短距離
    let _, fromStartCost = dijkstra edges nodeSet 1 |> Seq.find (fst >> (=) n)
    let _, fromEndCost = dijkstra edges nodeSet N |> Seq.find (fst >> (=) n)
    fromStartCost + fromEndCost)
  |> Seq.toList

In [14]:
solve [
  1, 2, 2
  1, 3, 3
  2, 5, 2
  3, 4, 1
  3, 5, 4
  4, 7, 5
  5, 6, 1
  5, 7, 6
  6, 7, 3
]

index,value
0,8
1,8
2,9
3,9
4,8
5,8
6,8


In [15]:
solve [
  1, 2, 1
  2, 3, 10
  3, 4, 100
]

index,value
0,111
1,111
2,111
3,111


In [16]:
solve [
  1, 2, 314
  1, 3, 159
  1, 4, 265
]

index,value
0,265
1,893
2,583
3,265


In [17]:
solve [
  1, 2, 479
  2, 3, 665
  3, 4, 154
  4, 5, 269
  5, 6, 501
  6, 7, 998
  7, 8, 992
  8, 9, 904
  9, 10, 763
  10, 11, 254
  11, 12, 591
  12, 13, 869
  13, 14, 843
  14, 15, 683
  15, 16, 708
  16, 17, 410
  17, 18, 88
  18, 19, 352
  19, 20, 566
  4, 8, 308
  3, 19, 573
  8, 14, 566
  10, 14, 766
  5, 16, 76
  3, 11, 887
  2, 6, 433
  5, 8, 570
  6, 15, 151
  4, 11, 737
  9, 17, 18
  4, 12, 900
  1, 17, 208
  15, 20, 973
  11, 15, 317
  13, 17, 260
  1, 7, 245
  6, 10, 38
  10, 12, 860
  2, 16, 442
  4, 20, 255
]

index,value
0,1214
1,1521
2,1526
3,1218
4,1218
5,1937
6,1704
7,1693
8,1250
9,2013
